### **Genetic Algo - Eight Queen Problem**
Details - https://en.wikipedia.org/wiki/Eight_queens_puzzle
* Import all required packages

In [217]:
import random
import numpy as np
from numpy.random import choice
import pandas as pd
from itertools import permutations

Define initial parameters required.
* n number of queens to be placed - in this case it is 8
* total Population - Number entries in initial population
* alpha list is a simple list of all possible numbers 0 to 7
* Population Permutation - List of all possible permutations for given n

In [218]:
n = 8
totalPopulation = 150
alpha_list = list(range(n))
Population_Permutation = list(map(list, list(permutations(alpha_list))))

Function to generate initial population

In [219]:
def Initial_Population(totalPopulation, n):
  Population = []
  secure_random = random.SystemRandom()
  for outloop in range(totalPopulation):
    selectedData = secure_random.choice(Population_Permutation)
    Population.append(selectedData)
  return Population

**Fitness Function**
* Function Diagonal Check - Performs validation of Queen placement with respect to it diagonal positions
* Function Score - Takes care of validation of Queen placement with respect to columns and Rows

Both together assigns a Fitness Score to each entry in Population.

Fitness is based on number of Queens placed in Non attack positions. if 3 Queens Out of 8 doesnt attack each other than score is 3.

In [220]:
def diagonal_check(samplematrix, i, j, n, direction):
  if i<n and j<n and direction == 315:
    if samplematrix[i+1][j+1] != 0:
      return -1
    else:
      return diagonal_check(samplematrix, i+1, j+1, n, direction=315)
  
  if i<n and j>0 and direction == 225:
    if samplematrix[i+1][j-1] != 0:
      return -1
    else:
      return diagonal_check(samplematrix, i+1, j-1, n, direction=225)
  
  if i>0 and j<n and direction == 45:
    if samplematrix[i-1][j+1] != 0:
      return -1
    else:
      return diagonal_check(samplematrix, i-1, j+1, n, direction=45)

  if i>0 and j>0 and direction == 135:
    if samplematrix[i-1][j-1] != 0:
      return -1
    else:
      return diagonal_check(samplematrix, i-1, j-1, n, direction=135)
  return 1

In [221]:
def Score(sample, n):
  samplescore = 0
  samplematrix = np.zeros([n,n],dtype=np.int8)
  for loop in range(n):
      samplematrix[sample[loop]][loop] = 1
  
  for loop in range(n):
    res = 4
    if samplematrix[sample[loop]].sum() != 1:
      res = -1
    else:  
      res = diagonal_check(samplematrix, sample[loop], loop, n-1, direction=45)
      res = res + diagonal_check(samplematrix, sample[loop], loop, n-1, direction=135)
      res = res + diagonal_check(samplematrix, sample[loop], loop, n-1, direction=225)
      res = res + diagonal_check(samplematrix, sample[loop], loop, n-1, direction=315)
    if res == 4:
      samplescore = samplescore + 1
  return samplescore

Scores / Assigness a Fitness Value for whole population provided to it.

In [222]:
def Population_Score(Population, n):
  PopScore = pd.DataFrame()
  PopScore["Population"] = Population
  res = []
  for loop in range(len(Population)):
    res.append(Score(Population[loop], n))
  PopScore["Score"] = res
  PopScore["PScore"] = (PopScore.Score / PopScore.Score.sum())
  return PopScore

**CrossOver**
* Pick any two traits of each chromosome and swap their values.

In [223]:
def Cross_Over(Selection1, Selection2, CrossOverRate, n):
  #return Selection1[:int(np.floor(n*CrossOverRate))] + Selection2[int(np.floor(n*CrossOverRate)):] 
  secure_random = random.SystemRandom()
  for loop in range(int(np.floor(n*CrossOverRate))):
    alpha = secure_random.choice(alpha_list)
    index1 = Selection1.index(alpha)
    index2 = Selection2.index(alpha)
    value1 = Selection1[index2]
    value2 = Selection2[index1]
    Selection1[index2] = alpha
    Selection2[index1] = alpha
    Selection1[index1] = value1
    Selection2[index2] = value2
  return Selection1, Selection2

**Mutation**
* Scramble Mutation concept is utilized here
* Pick any two positions randomly from a chromosome / single list and swap their positions.

In [224]:
def Mutation(Population, MutationRate):
  secure_random = random.SystemRandom()
  for loop in range(int(np.floor(len(Population)*MutationRate))):
    Selection = secure_random.choice(Population)
    index = Population.index(Selection)
    alpha1 = secure_random.choice(alpha_list)
    alpha2 = secure_random.choice(alpha_list)
    temp = Selection[alpha1]
    Selection[alpha1] = Selection[alpha2]
    Selection[alpha2] = temp
    Population[index] = Selection
  return Population

**Generate Offsprings**
* Selection and CrossOver
* Selection is done using Roulette Wheel method


In [225]:
def Generate_Offsprings(ScoredPopulation, totalPopulation, SelectionRate, CrossOverRate, n):
  offsprings = []
  ScoredPopulation = ScoredPopulation.sort_values('PScore', ascending=True)
  ScoredPopulation["PScoreCum"] = ScoredPopulation.PScore.cumsum()
  for loop in range(int(np.floor(totalPopulation*SelectionRate/2))):
    DrawNumber = random.random()
    Selection1 = ScoredPopulation[ScoredPopulation.PScoreCum > DrawNumber].head(1).Population
    DrawNumber = random.random()
    Selection2 = ScoredPopulation[ScoredPopulation.PScoreCum > DrawNumber].head(1).Population
    Selection1, Selection2 = Cross_Over(list(Selection1)[0], list(Selection2)[0], CrossOverRate, n)
    offsprings.append(Selection1)
    offsprings.append(Selection2)
  return offsprings

**Main Program** - Sequenced in following steps
* Generate Initial Population and Score / call Fitness Function
* Define Selection Rate, Cross Over Rate and Mutation Rate
* Loop upto 200 Generations till Fitness of 8 is reached.
  * Step 1 - Select and Generate Offsprings by CrossOver
  * Step 2 - Mutate Offsprings
  * Step 3 - Score Offsprings / Call Fitness Function
  * Step 4 - Check if desired Fitness level is reached. If yes then Break from loop otherwise repeat for next generation.

In [234]:
Population = Initial_Population(totalPopulation, n)
ScoredPopulation = Population_Score(Population, n)

In [235]:
SelectionRate = .05 # 5% of Population will be selected for Cross Over
CrossOverRate = .25 # 25% of 8 elements that 2 elements will be cross over
MutationRate = .10 # 10% of OffSprings

In [236]:
for loop in range(100):
  Offsprings = Generate_Offsprings(ScoredPopulation, totalPopulation, SelectionRate, CrossOverRate, n)
  Offsprings = Mutation(Offsprings, MutationRate)
  ScoredOffsprings = Population_Score(Offsprings, n)
  ScoredPopulation.drop(ScoredPopulation.head(int(np.floor(totalPopulation*SelectionRate))).index,inplace=True)
  ScoredPopulation = ScoredPopulation.append(ScoredOffsprings, ignore_index = True)
  maxscore = ScoredPopulation.Score.max()
  print("Gen - ", loop, "   TopScore - ", maxscore, "   Best in Population - ", list(ScoredPopulation[ScoredPopulation.Score == maxscore].head(1).Population)[0])
  if ScoredPopulation.Score.max() == n:
    break

Gen -  0    TopScore -  6    Best in Population -  [5, 3, 4, 7, 0, 2, 1, 6]
Gen -  1    TopScore -  6    Best in Population -  [5, 3, 4, 7, 0, 2, 1, 6]
Gen -  2    TopScore -  6    Best in Population -  [3, 6, 0, 1, 4, 7, 5, 2]
Gen -  3    TopScore -  6    Best in Population -  [3, 5, 7, 2, 4, 6, 1, 0]
Gen -  4    TopScore -  6    Best in Population -  [3, 5, 7, 2, 4, 6, 1, 0]
Gen -  5    TopScore -  6    Best in Population -  [3, 5, 7, 2, 4, 6, 1, 0]
Gen -  6    TopScore -  5    Best in Population -  [5, 1, 6, 2, 7, 4, 0, 3]
Gen -  7    TopScore -  5    Best in Population -  [5, 1, 6, 2, 7, 4, 0, 3]
Gen -  8    TopScore -  5    Best in Population -  [5, 1, 6, 2, 7, 4, 0, 3]
Gen -  9    TopScore -  5    Best in Population -  [2, 6, 3, 1, 5, 4, 0, 7]
Gen -  10    TopScore -  5    Best in Population -  [0, 6, 1, 3, 5, 4, 2, 7]
Gen -  11    TopScore -  5    Best in Population -  [0, 6, 1, 3, 5, 4, 2, 7]
Gen -  12    TopScore -  5    Best in Population -  [0, 5, 4, 3, 6, 1, 2, 7]
Gen -  13